<a href="https://colab.research.google.com/github/marcinwolter/DNN_examples/blob/master/dnn_tracking_3D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
'''
Reconstructs straight tracks in 3D (with noise and inefficiency)
Using google TPUs to speed up the training
'''

from __future__ import print_function

!pip install keras_sequential_ascii

#import ROOT
#from ROOT import TMVA, TFile, TTree, TCut, TChain, TString, TF1, TGraphErrors, TFile, TNtuple, TCanvas


import matplotlib.pyplot as plt
# Package imports
import numpy as np
from scipy.stats import norm

import tensorflow as tf

import os
import pprint


from keras import backend as K
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, RepeatVector, LSTM, TimeDistributed
from keras.layers import Conv3D, MaxPooling3D
from keras.layers import Reshape
from keras.models import Sequential
from keras_sequential_ascii import sequential_model_to_ascii_printout
from keras.utils import plot_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import load_model
from keras.optimizers import SGD, RMSprop, Adam, Nadam

import sys
import time

np.random.seed(2348)

batch_size = 5000
batch_size_NN = 2*128 #128
epochs = 512 #512
pre_epochs =  16 #32 epochs for pre-traing using rms optimizer
post_epochs = 16 #64 post-training using SGD
steps_per_epoch = 2*8 #4*8

# input image dimensions
img_rows, img_cols = 28, 28  #28, 28


####################################################################################################

# Data parameters
det_width = img_rows  # stupid, to be changed
det_depth = img_cols
det_shape = (det_width, det_width, det_depth)


# Number of tracks in each event follows Poisson distribution
mean_tracks = 2
max_tracks  = 2

#probability of noise and the efficiency<1
prob_noise = 0.000
efficiency = 0.90

#slope multiplicator - reduce the slope to slope_mult*max_slope
slope_mult = 0.4


y_new = []

# ### Functions for toy data generation


def simulate_straight_track(b2x, bx, b2y, by, det_shape):
    """
    Simulate detector data for one straight track.
    Parameters:
        b2x, b2y: track last layer intercept
        bx, by: track first-layer intercept parameter (detector entry point)
        det_shape: tuple of detector shape: (width, width, depth)
    Returns:
        ndarray of binary detector data for one track.
    """
    x = np.zeros(det_shape)
    idz = np.arange(det_shape[2])
    hitsx = (idz*(b2x-bx)/det_depth + bx +0.5).astype(int)  ## +0.5 to get rounding, not clipping
    hitsy = (idz*(b2y-by)/det_depth + by +0.5).astype(int)  ## +0.5 to get rounding, not clipping
    # implement hit efficiency
    for i in range(det_shape[0]):
        if (np.random.random() > efficiency):
            hitsx[i]=0
            hitsy[i]=0
    #print ("HITS = ",hits)
    valid = (hitsx > 0) & (hitsx < det_shape[0]) & (hitsy > 0) & (hitsy < det_shape[1])
    x[hitsx[valid], hitsy[valid], idz[valid]] = 1
    return x

# Generator for single-track events
def gen_tracks(batch_size=batch_size, det_shape=det_shape):
    """Arguments:
         batch_size: number of events to yield for each call
       Yields: batches of training data for use with the keras fit_generator function
    """
    while True:
        # Entry and exit points are randomized
        bsx = np.random.random_sample(size=batch_size)*(det_width-6)+3
        b2sx = np.random.random_sample(size=batch_size)*(det_width-6)+3
        # restrict the slope of a track to slope_mult*slope
        b2sx = bsx + slope_mult*(b2sx-bsx)
        
        bsy = np.random.random_sample(size=batch_size)*(det_width-6)+3
        b2sy = np.random.random_sample(size=batch_size)*(det_width-6)+3
        # restrict the slope of a track to slope_mult*slope
        b2sy = bsy + slope_mult*(b2sy-bsy)
        
        tracks = np.zeros((batch_size, 1, det_depth, det_width, det_width))
        params = zip(bsx, b2sx, bsy, b2sy )
        for i, (pbx, pb2x, pby, pb2y) in enumerate(params):
            tracks[i,0] = simulate_straight_track(pb2x, pbx, pb2y, pby, det_shape)
###        targets = zip(bsx, msx, bsy, msy)
        targets = zip(bsx/det_width, (b2sx-bsx)/det_depth/slope_mult, 
                      bsy/det_width, (b2sy-bsy)/det_depth/slope_mult )  # save b and b2 as targets
        targets = np.asarray(targets)
        yield tracks, targets

# Generator for multi-track events.
# Each event contains exactly n_tracks tracks.
# The target track parameters are sorted in increasing order of X intercept.
def gen_n_tracks(batch_size=batch_size, det_shape=det_shape, n_tracks=mean_tracks):
    gen_single = gen_tracks(batch_size=n_tracks, det_shape=det_shape)
    while True:
        batch_events = np.zeros((batch_size, 1, det_depth, det_width, det_width))
#        batch_targets = -np.ones((batch_size, n_tracks, 4))
        batch_targets = -np.ones((batch_size, max_tracks, 4))
        #print("batch_targets 0 ",batch_targets.shape)
        for n in range(batch_size):
            tracks,targets = gen_single.next()
            batch_events[n,0] = np.clip( sum( tracks ), 0, 1)
            event_targets = np.asarray(targets)

            #print("event_targets",event_targets,targets)
            batch_targets[n] = event_targets[event_targets[:,0].argsort()] # sort by first column
            #add empty rows to get the size of max_tracks
            #for k in range(max_tracks-n_tracks):
            ###batch_targets[n] = event_targets  #do not sort
            #print("batch_events ",batch_events.shape)
            #print("batch_targets ",batch_targets.shape)
        yield batch_events, batch_targets

# Generator for multi-track events.
# Each event contains up to max_tracks tracks.
# The target track parameters are sorted in increasing order of intercept.
def gen_multi_tracks(batch_size=batch_size, det_shape=det_shape, mean_tracks=mean_tracks):
    gen_single = gen_tracks(batch_size=max_tracks, det_shape=det_shape)
    while True:
        batch_events = np.zeros((batch_size, 1, det_depth, det_width, det_width))
        batch_targets = -np.ones((batch_size, max_tracks, 4))
        for n in range(batch_size):
            num_tracks = min( max_tracks, np.random.poisson(mean_tracks) )
            tracks,targets = gen_single.next()
            batch_events[n,0] = np.clip( sum( tracks[:num_tracks] ), 0, 1)
            event_targets = np.asarray(targets[:num_tracks])
            batch_targets[n,:num_tracks] = event_targets[event_targets[:,0].argsort()] # sort by first column            
        yield batch_events, batch_targets


def gen_noise(batch_size=batch_size, det_shape=det_shape, prob_noise=prob_noise):

        batch_events = np.zeros((batch_size, 1, det_depth, det_width, det_width))
        for n in range(batch_size):

            for i in range(det_depth):
               for j in range(det_width):
                  for k in range(det_width):
                     if np.random.random()<prob_noise:
                        batch_events[n,0,i,j,k]=1


        yield batch_events


####################################################################################################





##########################################################################################

def get_activations(model, model_inputs, print_shape_only=False, layer_name=None):
    print('----- activations -----')
    activations = []
    inp = model.input

    model_multi_inputs_cond = True
    if not isinstance(inp, list):
        # only one input! let's wrap it in a list.
        inp = [inp]
        model_multi_inputs_cond = False

    outputs = [layer.output for layer in model.layers if
               layer.name == layer_name or layer_name is None]  # all layer outputs



    funcs = [K.function(inp + [K.learning_phase()], [out]) for out in outputs]  # evaluation functions

    if model_multi_inputs_cond:
        list_inputs = []
        list_inputs.extend(model_inputs)
        list_inputs.append(0.)
    else:
        list_inputs = [model_inputs, 0.]

    # Learning phase. 0 = Test mode (no dropout or batch normalization)
    # layer_outputs = [func([model_inputs, 0.])[0] for func in funcs]
    layer_outputs = [func(list_inputs)[0] for func in funcs]
    for layer_activations in layer_outputs:
        activations.append(layer_activations)
        if print_shape_only:
            print(layer_activations.shape)
        else:
            print(layer_activations)
    return activations




#####################################################################################


In [0]:

# the data, split between train and test sets
#(x_train, y_train), (x_test, y_test) = mnist.load_data()


###x_train, y_train = gen_n_tracks(batch_size=batch_size).next()
x_test, y_test = gen_n_tracks(batch_size=batch_size, n_tracks=mean_tracks).next()

###y_train = y_train
###y_test = y_test

# add noise
###noise_train = gen_noise(batch_size=batch_size).next()
noise_test = gen_noise(batch_size=batch_size, prob_noise=prob_noise).next()
###x_train = x_train+noise_train
x_test  = x_test+noise_test


###print("x_train ",x_train.shape)
###print("y_train ",y_train.shape)




#print(x_train[0,0])
#print("Train targets ",y_train[0])
#print(x_test[0,0])
#print("Test targets ",y_test[0])


#
# if K.image_data_format() == 'channels_first':
#     x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
#     x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
#     y_train = y_train.reshape(y_train.shape[0], 1, 2)
#     y_test = y_test.reshape(y_test.shape[0], 1, 2)
#     input_shape = (1, img_rows, img_cols)
# else:
#     x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
#     x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
#     y_train = y_train.reshape(y_train.shape[0], 2, 1)
#     y_test = y_test.reshape(y_test.shape[0], 2, 1)
#     input_shape = (img_rows, img_cols, 1)

###x_train = x_train.astype('int')
x_test = x_test.astype('int')
###x_train = np.clip(x_train, 0, 1)
x_test = np.clip(x_test, 0, 1)
#x_train /= 255
#x_test /= 255
#print('x_train shape:', x_train.shape)
#print(x_train.shape[0], 'train samples')
#print(x_test.shape[0], 'test samples')
#print('y_train shape:', y_train.shape)
#print(y_train.shape[0], 'train samples')
#print(y_test.shape[0], 'test samples')

In [0]:
#plot sample events

#for n in range(min(16,len(x_train))):
#  plt.subplot(4, 4, n+1)
#  plt.imshow(x_train[n,0])


for n in range(1):
  for k in range(int(100*(prob_noise+0.10))):
    rnumber = np.random.randint(0,len(x_test))
    x,y,z = x_test[rnumber,0].nonzero()
    from mpl_toolkits.mplot3d import Axes3D
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    ax.scatter(x, y, z, zdir='z', c= 'red', s=5, alpha=0.9)
    
    
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z (depth)')
  
    
    for j in range(len(y_test[rnumber])):
    # Data for a three-dimensional line
       #print("index ",k,j)
       #print("ytrain ", y_train[rnumber])
       zline = np.linspace(0, 28, 100)
       xline = y_test[rnumber,j,0]*det_width+y_test[rnumber,j,1]*slope_mult*zline 
       yline = y_test[rnumber,j,2]*det_width+y_test[rnumber,j,3]*slope_mult*zline 
###       xline = 100*y_test[rnumber,k,0]+100/slope_scale*y_test[rnumber,k,1]*zline
###       yline = 100*y_test[rnumber,k,2]+100/slope_scale*y_test[rnumber,k,3]*zline
       ax.plot3D(xline, yline, zline,  'green', alpha=0.5)
    
    ax.view_init(elev=35., azim=45)
    ax.scatter([0,det_width],[0,det_width],[0,det_depth], c= 'blue', s=0)

    # set axes range
    ax.set_xlim3d(0,det_width)
    ax.set_xlim3d(0,det_width)
    ax.set_xlim3d(0,det_depth)
    #ax.axis('equal')
    
    print('event_'+str(efficiency)+"_"+str(prob_noise)+"_"+str(k)+'.png')
    plt.savefig('event_'+str(efficiency)+"_"+str(prob_noise)+"_"+str(k)+'.png')
    plt.show()
  
#  plt.imshow(x_train[np.random.randint(0,len(x_train)),0])
#  plt.show()
#   plt.savefig('event.png')
#plt.clf()





#for i in range(img_rows):
#    print (" ")
#    for j in range (img_cols):
#        print (x_train[0,0,i,j], end=' ')
#print (" ")
#print ("y_train[0] = ",y_train[0])
#print ("y_test[0] = ",y_test[0])


# convert class vectors to binary class matrices
#y_train = keras.utils.to_categorical(y_train, num_classes)
#y_test = keras.utils.to_categorical(y_test, num_classes)
#print ("Class = ",y_train[0])



#sys.exit("Stopping here")



In [0]:
#printing TPU information

if 'COLAB_TPU_ADDR' not in os.environ:
  print('ERROR: Not connected to a TPU runtime; please see the first cell in this notebook for instructions!')
else:
  tpu_address = 'grpc://' + os.environ['COLAB_TPU_ADDR']
  print ('TPU address is', tpu_address)

  with tf.Session(tpu_address) as session:
    devices = session.list_devices()
    
  print('TPU devices:')
  pprint.pprint(devices)

In [0]:



start_training_time = time.time()

if 'COLAB_TPU_ADDR' not in os.environ:

  model = Sequential()
  input_shape=(1,det_width, det_width, det_depth)

  print("input_shape ",input_shape)

  model.add(Reshape((det_width, det_width, det_depth,1),
                                  input_shape=input_shape))
  model.add(Conv3D(32, (3, 3, 3), activation='relu'))  #8
  model.add(Conv3D(32, (3, 3, 3), activation='relu'))  #8
#  model.add(Conv3D(32, (3, 3, 3), activation='relu'))  #added
  model.add(MaxPooling3D(pool_size=(2, 2, 2)))
  model.add(Conv3D(128, (3, 3, 3), activation='relu'))  #32
  model.add(Conv3D(128, (3, 3, 3), activation='relu'))  #32
#  model.add(Conv3D(128, (3, 3, 3), activation='relu'))  #added
  model.add(Dropout(0.25))
  model.add(Flatten())
  model.add(Dense(400, activation='relu'))  # 400
#  model.add(Dense(400))  # added
  #model.add(Dropout(0.5))
  #model.add(Dense(2, activation='tanh'))


  model.add(RepeatVector(max_tracks))
  model.add(LSTM(400, return_sequences=True))
  model.add(TimeDistributed(Dense(4)))

  # model.add(Flatten(input_shape=input_shape))
  # #model.add(Reshape(input_shape - (1, ), input_shape=input_shape))
  # model.add(Dense(32, activation='relu'))
  # model.add(Dense(32, activation='relu'))
  # model.add(Dense(32, activation='relu'))
  # model.add(Dense(32, activation='relu'))
  # #model.add(Dense(32, activation='relu'))
  # #model.add(Dense(32, activation='relu'))
  # model.add(Dropout(0.5))
  # model.add(Dense(2, activation='tanh'))


  #model.add(Reshape((1, 2),input_shape=(2,)))
  model.add(Reshape((max_tracks, 4),input_shape=(4*max_tracks,)))

else:

  model = tf.keras.layers.Sequential()
  input_shape=(1,det_width, det_width, det_depth)

  print("input_shape ",input_shape)

  model.add(Reshape((det_width, det_width, det_depth,1),
                                  input_shape=input_shape))
  model.add(Conv3D(32, (3, 3, 3), activation='relu'))  #8
  model.add(tf.keras.layers.Conv3D(32, (3, 3, 3), activation='relu'))  #8
  model.add(tf.keras.layers.Conv3D(32, (3, 3, 3), activation='relu'))  #added
  model.add(tf.keras.layers.MaxPooling3D(pool_size=(2, 2, 2)))
  model.add(tf.keras.layers.Conv3D(128, (3, 3, 3), activation='relu'))  #32
  model.add(tf.keras.layers.Conv3D(128, (3, 3, 3), activation='relu'))  #32
  model.add(tf.keras.layers.Conv3D(128, (3, 3, 3), activation='relu'))  #added
  model.add(tf.keras.layers.Dropout(0.25))
  model.add(tf.keras.layers.Flatten())
  model.add(tf.keras.layers.Dense(400, activation='relu'))  # 400
  model.add(tf.keras.layers.Dense(400))  # added
  #model.add(Dropout(0.5))
  #model.add(Dense(2, activation='tanh'))


  model.add(tf.keras.layers.RepeatVector(max_tracks))
  model.add(tf.keras.layers.LSTM(400, return_sequences=True))
  model.add(tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(4)))

  # model.add(Flatten(input_shape=input_shape))
  # #model.add(Reshape(input_shape - (1, ), input_shape=input_shape))
  # model.add(Dense(32, activation='relu'))
  # model.add(Dense(32, activation='relu'))
  # model.add(Dense(32, activation='relu'))
  # model.add(Dense(32, activation='relu'))
  # #model.add(Dense(32, activation='relu'))
  # #model.add(Dense(32, activation='relu'))
  # model.add(Dropout(0.5))
  # model.add(Dense(2, activation='tanh'))


  #model.add(Reshape((1, 2),input_shape=(2,)))
  model.add(tf.keras.layers.Reshape((max_tracks, 4),input_shape=(4*max_tracks,)))  
  

# print outputs for each layer
outputs =  [layer.output for layer in model.layers]
print (outputs)
print (model.summary())

print ("plotting model")
plot_model(model, to_file='model.png',rankdir='LR',show_shapes=False,show_layer_names=False )  ###, show_shapes=True

print ("Showing model")
from IPython.display import Image
display(Image(filename='model.png'))
!ls
print ("Model shown")

#sys.exit("Stopping here")



'''
    Converts a Keras model to dot format and save to a file.
    # Arguments
        model: A Keras model instance
        to_file: File name of the plot image.
        show_shapes: whether to display shape information.
        show_layer_names: whether to display layer names.
        rankdir: `rankdir` argument passed to PyDot,
            a string specifying the format of the plot:
            'TB' creates a vertical plot;
            'LR' creates a horizontal plot.
        expand_nested: whether to expand nested models into clusters.
        dpi: dot DPI.
'''

# Vizualizing model structure

sequential_model_to_ascii_printout(model)


#history=model.fit(x_train, y_train,
#          batch_size=batch_size_NN,
#          epochs=epochs,
#          verbose=1,
#          validation_data=(x_test, y_test))


def data_generator(batch_size_gen=batch_size_NN,prob_noise=prob_noise,n_tracks=mean_tracks):
  
 while True: 
   # the data, split between train and test sets
   #(x_train, y_train), (x_test, y_test) = mnist.load_data()

   x_train, y_train = gen_n_tracks(batch_size=batch_size_gen,n_tracks=mean_tracks).next()

   #y_train = y_train/100

   # add noise
   noise_train = gen_noise(batch_size=batch_size_gen,prob_noise=prob_noise).next()
   x_train = x_train+noise_train
   
   #print("x_train: ",x_train.shape)
   #print("y_train: ",y_train.shape)
   yield x_train, y_train
   
#print("generator ",data_generator())



In [0]:

#using TPU model!!!
# This address identifies the TPU we'll use when configuring TensorFlow.
if 'COLAB_TPU_ADDR'  in os.environ:  
  TPU_WORKER = 'grpc://' + os.environ['COLAB_TPU_ADDR']
  tf.logging.set_verbosity(tf.logging.INFO)

  model = tf.contrib.tpu.keras_to_tpu_model(
     model,
     strategy=tf.contrib.tpu.TPUDistributionStrategy(
        tf.contrib.cluster_resolver.TPUClusterResolver(TPU_WORKER)))

  model.compile(optimizer=tf.train.AdamOptimizer(), loss='mse')
else:
#  model.compile(
#      optimizer=Adam(lr=0.001, beta_1=0.9, beta_2=0.999, 
#                                      epsilon=1e-8), loss='mse')
#  rms = RMSprop(lr=0.001, rho=0.9, epsilon=None, decay=0.1)
#  model.compile(optimizer=rms, loss='mse')
#  model.compile(optimizer='Adam', loss='mean_squared_error')
  model.compile(optimizer='sgd', loss='mean_squared_error')

#model.compile(optimizer=tf.train.AdamOptimizer(), loss='mse')
#model.compile(loss='mean_squared_error', optimizer='Adam')

#model.compile(loss=keras.losses.categorical_crossentropy,
#              optimizer=keras.optimizers.Adadelta(),
#              metrics=['accuracy'])


In [0]:
#del model
#model = load_model('partly_trained.h5')

# patient early stopping
es = EarlyStopping(monitor='loss', mode='min', verbose=1, patience=12)  # 'val_loss'
mc = ModelCheckpoint('best_model.h5', monitor='loss', mode='min', verbose=1,
                     save_best_only=True)  # 'val_loss'

#train with fit_generator - generate events on flight
# pre-training with rms optimizer 
history=model.fit_generator(data_generator(prob_noise=prob_noise, n_tracks=mean_tracks),
                            steps_per_epoch=steps_per_epoch,
                            #validation_data=data_generator(), 
                            #validation_steps=steps_per_epoch,
                            epochs = pre_epochs, callbacks=[es, mc])



In [0]:
# summarize history for loss
plt.plot(history.history['loss'])
###plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.xlim(3, None)
plt.ylim(ymax = 1.2*history.history['loss'][min(len(history.history['loss']),3)-1], ymin = 0)
print("history",history.history['loss'])

auxName = 'pretraining_'+str(efficiency)+'_'+str(prob_noise)+'.png'
plt.savefig(auxName)
  
plt.show()
#plt.savefig('training.png')
plt.clf()

In [0]:


def train_model(model, prob_noise=prob_noise):

  print("Training with ADAM optimizer, efficiency, prob_noise = ",efficiency,prob_noise)
  
  #recompile the model with Adam optimizer 
  model.compile(optimizer='Adam', loss='mean_squared_error')

  # patient early stopping
  es_noise = EarlyStopping(monitor='loss', mode='min', verbose=1, patience=12)  # 'val_loss'
  mc_noise = ModelCheckpoint('best_model.h5', monitor='loss', mode='min', verbose=1,
                     save_best_only=True)  # 'val_loss'


  #train with fit_generator - generate events on flight
  history=model.fit_generator(data_generator(prob_noise=prob_noise, n_tracks=mean_tracks),
                            steps_per_epoch=steps_per_epoch,
                            #validation_data=data_generator(), 
                            #validation_steps=steps_per_epoch,
                            epochs = epochs, callbacks=[es_noise, mc_noise])


  # load the saved model
  model = load_model('best_model.h5')

  # summarize history for loss
  plt.plot(history.history['loss'])
  ###plt.plot(history.history['val_loss'])
  plt.title('model loss')
  plt.ylabel('loss')
  plt.xlabel('epoch')
  plt.legend(['train', 'test'], loc='upper left')
  plt.xlim(3, None)
  plt.ylim(ymax = 1.2*history.history['loss'][min(len(history.history['loss']),3)-1], ymin = 0)
  print("history",history.history['loss'])

  auxName = 'training_'+str(efficiency)+'_'+str(prob_noise)+'.png'
  plt.savefig(auxName)
  
  plt.show()
  plt.clf()
  
  
    #recompile the model with SGD optimizer 
  model.compile(optimizer='SGD', loss='mean_squared_error')

  

  #train with fit_generator - generate events on flight
  history=model.fit_generator(data_generator(prob_noise=prob_noise, n_tracks=mean_tracks),
                            steps_per_epoch=steps_per_epoch,
                            #validation_data=data_generator(), 
                            #validation_steps=steps_per_epoch,
                            epochs = post_epochs, callbacks=[es_noise, mc_noise])



  # load the saved model
  model = load_model('best_model.h5')
  
  # summarize history for loss
  plt.plot(history.history['loss'])
  ###plt.plot(history.history['val_loss'])
  plt.title('model loss')
  plt.ylabel('loss')
  plt.xlabel('epoch')
  plt.legend(['train', 'test'], loc='upper left')
  plt.xlim(3, None)
  plt.ylim(ymax = 1.2*history.history['loss'][min(len(history.history['loss']),3)-1], ymin = 0)
  print("history",history.history['loss'])

  auxName = 'trainingSGD_'+str(efficiency)+'_'+str(prob_noise)+'.png'
  plt.savefig(auxName)
  
  plt.show()
  plt.clf()

In [0]:
def generate_testdata(prob_noise = prob_noise):

  print("Generate test data, efficiency, prob_noise = ",efficiency,prob_noise)

  ###x_train, y_train = gen_n_tracks(batch_size=batch_size).next()
  x_test, y_test = gen_n_tracks(batch_size=batch_size, n_tracks=mean_tracks).next()

  # add noise
  noise_test = gen_noise(batch_size=batch_size, prob_noise=prob_noise).next()
  x_test  = x_test+noise_test
 

  x_test = x_test.astype('int')
  #clip to 1
  x_test = np.clip(x_test, 0, 1)

  return x_test, y_test

In [0]:

def plot_sample():

  #plot sample events
  print("plotting sample events, efficiency, prob_noise = ",efficiency,prob_noise)

  for n in range(1):
    for k in range(int(100*(prob_noise+0.10))):
      rnumber = np.random.randint(0,len(x_test))
      x,y,z = x_test[rnumber,0].nonzero()
      from mpl_toolkits.mplot3d import Axes3D
      fig = plt.figure()
      ax = fig.add_subplot(111, projection='3d')
      ax.scatter(x, y, z, zdir='z', c= 'red', s=5, alpha=0.9)
    
    
      ax.set_xlabel('X')
      ax.set_ylabel('Y')
      ax.set_zlabel('Z (depth)')
  
    
      for j in range(len(y_test[rnumber])):
      # Data for a three-dimensional line
         #print("index ",k)
         #print("ytrain ", y_train[rnumber])
         zline = np.linspace(0, 28, 100)
         xline = y_test[rnumber,j,0]*det_width+y_test[rnumber,j,1]*slope_mult*zline 
         yline = y_test[rnumber,j,2]*det_width+y_test[rnumber,j,3]*slope_mult*zline 
###       xline = 100*y_test[rnumber,k,0]+100/slope_scale*y_test[rnumber,k,1]*zline
###       yline = 100*y_test[rnumber,k,2]+100/slope_scale*y_test[rnumber,k,3]*zline
         ax.plot3D(xline, yline, zline,  'green', alpha=0.5)
    
      ax.view_init(elev=35., azim=45)
      ax.scatter([0,det_width],[0,det_width],[0,det_depth], c= 'blue', s=0)

      # set axes range
      ax.set_xlim3d(0,det_width)
      ax.set_xlim3d(0,det_width)
      ax.set_xlim3d(0,det_depth)
      #ax.axis('equal')

      print('event_'+str(efficiency)+"_"+str(prob_noise)+"_"+str(k)+'.png')
      plt.savefig('event_'+str(efficiency)+"_"+str(prob_noise)+"_"+str(k)+'.png')
      plt.show()
  
#  plt.imshow(x_train[np.random.randint(0,len(x_train)),0])
#  plt.show()
#plt.savefig('event.png')
#plt.clf()






#sys.exit("Stopping here")



In [0]:
def predict(model):

  print("Predict with trained NN, efficiency, prob_noise = ",efficiency,prob_noise)

  start_pattern_time = time.time()

  if 'COLAB_TPU_ADDR'  in os.environ:
  # converting tpu_model to the cpu_model
   model = model.sync_to_cpu()


# make a prediction
#batch_size=2
  print("x_test.shape", x_test.shape)

  y_new = model.predict(x_test)

  print (model.summary())
  print("y_new.shape", y_new.shape)

  end_pattern_time = time.time()

  return y_new

In [0]:


'''
# fit tracks using ROOT fit
y0_fit = []
for i in range(len(x_test)):
    y0_row = []

    grxzv = []
    xzfitv = []
    xztruev = []
    xzdnnv = []
    

    for j in range(len(y_new[i])):
        if i<30:
            print ("true  x0, Ax  ", (y_test[i])[j])
            print ("pred  x0, Ax  ", (y_new[i])[j])



            for m in range(img_rows):
              print (" ")
              for n in range (img_cols):
                 if abs(n - ( (((y_new[i])[j])[1]*100/slope_scale*m) + ((y_new[i])[j])[0]*100 ) ) < 4:
                   print (x_test[i,0,m,n]+5, end=' ')
                 else:
                   print (x_test[i,0,m,n], end=' ')
            print(" ")
            #print(" value at x=14 : ",(((y_new[i])[j])[1]*100/slope_scale*14) + ((y_new[i])[j])[0]*100 )

        N = 0
        x = []
        y = []
        x_err = []
        y_err = []

        for m in range(img_rows):
            for n in range (img_cols):
                if abs(n - ( (((y_new[i])[j])[1]*100/slope_scale*m) + ((y_new[i])[j])[0]*100 ) ) < 4:
                  #if i<10:
                  #   print (" i,m,n x_test ",i,m,n,x_test[i,0,m,n])
                  if x_test[i,0,m,n] == 1:
                      N = N+1
                      x.append(m)
                      y.append(n)
                      x_err.append(0.3)
                      y_err.append(0.3)
        if i<10:
          print ("N ", N) 
          print ("x vector ", x) 
          print ("y vector ", y) 

        if N>13:
           xzfit = TF1("xzfit","pol1",0,27)
           ###xzfit2 = TF1("xzfit2","pol1",0,27)
           grxz  = TGraphErrors(N,np.array(x,'d'),np.array(y,'d'), np.array(x_err,'d'), np.array(y_err,'d') )
           grxz.SetMarkerColor(ROOT.kRed)
           grxz.SetLineColor(ROOT.kRed)
           #grxz.Print()

           err = grxz.Fit(xzfit," +rob=0.70,q ")
           ##err = grxz.Fit(xzfit2,"  ")
           #print ("status ",err)
           x0 = xzfit.GetParameter(0)/100.
           Ax = xzfit.GetParameter(1)*slope_scale/100.
           chi2 = xzfit.GetChisquare()
           NDF = xzfit.GetNDF()
           prob = xzfit.GetProb()

           grxzv.append(grxz)
           xzfitv.append(xzfit)
           
           y0_row.append([x0,Ax,chi2,NDF,prob,N,xzfit.GetNumberFitPoints()])
           if i<30:
             print ("fit x0, Ax  ", x0, Ax)
           #del xzfit
           #del grxz
           #print ("Points: ",N, xzfit.GetNumberFitPoints())


              ###grxz.Draw("aps")
              #print("MWMW ",(y_test[i])[0])
              #print("MWMW ",((y_test[i])[0])[0])
           if len(y_new[i])>0:
                xztrue = TF1("xztrue","pol1",0,27)
                #print ("true track: ",((y_test[i])[0])[0], ((y_test[i])[0])[1] )
                xztrue.SetParameter(0, float(((y_test[i])[j])[0]*100.) )
                xztrue.SetParameter(1, float(((y_test[i])[j])[1]*100./slope_scale) )
                xztrue.SetLineColor(ROOT.kGreen)

                xzdnn = TF1("xzdnn","pol1",0,27)
                #print ("dnn track: ",((y_new[i])[0])[0], ((y_new[i])[0])[1] )
                xzdnn.SetParameter(0, float(((y_new[i])[j])[0]*100.) )
                xzdnn.SetParameter(1, float(((y_new[i])[j])[1]*100./slope_scale) )
                xzdnn.SetLineColor(ROOT.kBlue)


                xzdnnv.append(xzdnn)
                xztruev.append(xztrue)
                ##xzdnn.Draw("same")
                ##xztrue.Draw("same")



           
        else:
           y0_row.append([-999,-999,-999,-999,-999,-999,-999])

    if i<150:   
       MyCanv =  TCanvas( "MyCanv", "MyCanv", 800, 600 )
       legend = ROOT.TLegend(0.75, 0.75, 0.95, 0.95)
       Na = 0
       xa = []
       ya = []
       xa_err = []
       ya_err = []

       for m in range(img_rows):
            for n in range (img_cols):
                  if x_test[i,0,m,n] == 1:
                      Na = Na+1
                      xa.append(m)
                      ya.append(n)
                      xa_err.append(0.3)
                      ya_err.append(0.3)
       grxza  = TGraphErrors(Na,np.array(xa,'d'),np.array(ya,'d'), np.array(xa_err,'d'), np.array(ya_err,'d') )
       grxza.Draw("ap")
       grxza.GetXaxis().SetRangeUser(-0.5,27.5)
       grxza.GetYaxis().SetRangeUser(-0.5,27.5)
       grxza.Draw("ap")
       MyCanv.Update()

       legend.AddEntry(grxza, "hits", "lp")
       if len(grxzv)>0:
        legend.AddEntry(grxzv[0], "assoc. hits", "lp")
        legend.AddEntry(xzfitv[0], "fitted track", "lp")
        legend.AddEntry(xzdnnv[0], "dnn track", "lp")
        legend.AddEntry(xztruev[0], "true track", "lp")
        legend.Draw()

       for k in range(len(grxzv)):  
            grxzv[k].Draw("p same")
            xzfitv[k].Draw("same") 
            xzdnnv[k].Draw("same") 
            xztruev[k].Draw("same") 

       ttt = "images/fit_"+str(i)+".png"
       print ("saving ",i, ttt)
       MyCanv.SaveAs(ttt)
       MyCanv.Close()

    y0_fit.append(y0_row)

end_fit_time = time.time()
'''


'''
# Fill ntuple
ofn = 'aptuple.root'

outfile = TFile( ofn, 'RECREATE', 'ROOT file with an NTuple' )
outfile.cd()

#ntuple  = TNtuple( 'ntuple', 'pattern recognition') #'x0_true:Ax_true:x0_reco1:Ax_reco1:x0_dnn1:Ax_dnn1:x0_reco2:Ax_reco2:x0_dnn2:Ax_dnn2:x0_reco3:Ax_reco3:x0_dnn3:Ax_dnn3' )

#for i in range(len(y0_fit)):
    #words = string.split( line )
    #row = map( float, words )
#    if len(y_test[i])+len(y0_fit[i])+len(y_new[i])<
#       apply( ntuple.Fill, np.array(np.append(y_test[i],[y0_fit[i],y_new[i]])))
#ntuple.Write()


x0_true1 = np.array([0.],'f')
x0_true2 = np.array([0.],'f')
x0_true3 = np.array([0.],'f')
Ax_true1 = np.array([0.],'f')
Ax_true2 = np.array([0.],'f')
Ax_true3 = np.array([0.],'f')

x0_fit1 = np.array([0.],'f')
x0_fit2 = np.array([0.],'f')
x0_fit3 = np.array([0.],'f')
Ax_fit1 = np.array([0.],'f')
Ax_fit2 = np.array([0.],'f')
Ax_fit3 = np.array([0.],'f')

chi2_fit1 = np.array([0.],'f')
chi2_fit2 = np.array([0.],'f')
chi2_fit3 = np.array([0.],'f')
NDF_fit1 = np.array([0.],'f')
NDF_fit2 = np.array([0.],'f')
NDF_fit3 = np.array([0.],'f')
prob_fit1 = np.array([0.],'f')
prob_fit2 = np.array([0.],'f')
prob_fit3 = np.array([0.],'f')
N_fit1 = np.array([0.],'f')
N_fit2 = np.array([0.],'f')
N_fit3 = np.array([0.],'f')
Nfit_fit1 = np.array([0.],'f')
Nfit_fit2 = np.array([0.],'f')
Nfit_fit3 = np.array([0.],'f')


x0_dnn1 = np.array([0.],'f')
x0_dnn2 = np.array([0.],'f')
x0_dnn3 = np.array([0.],'f')
Ax_dnn1 = np.array([0.],'f')
Ax_dnn2 = np.array([0.],'f')
Ax_dnn3 = np.array([0.],'f')

tree = TTree( 'ntuple', 'pattern recognition' )

tree.Branch( 'x0_true1', x0_true1, 'x0_true1/F' )
tree.Branch( 'x0_true2', x0_true2, 'x0_true2/F' )
tree.Branch( 'x0_true3', x0_true3, 'x0_true3/F' )
tree.Branch( 'Ax_true1', Ax_true1, 'A0_true1/F' )
tree.Branch( 'Ax_true2', Ax_true2, 'A0_true2/F' )
tree.Branch( 'Ax_true3', Ax_true3, 'A0_true3/F' ) 

tree.Branch( 'x0_fit1', x0_fit1, 'x0_fit1/F' )
tree.Branch( 'x0_fit2', x0_fit2, 'x0_fit2/F' )
tree.Branch( 'x0_fit3', x0_fit3, 'x0_fit3/F' )
tree.Branch( 'Ax_fit1', Ax_fit1, 'A0_fit1/F' )
tree.Branch( 'Ax_fit2', Ax_fit2, 'A0_fit2/F' )
tree.Branch( 'Ax_fit3', Ax_fit3, 'A0_fit3/F' ) 

tree.Branch( 'chi2_fit1', chi2_fit1, 'chi2_fit1/F' )
tree.Branch( 'chi2_fit2', chi2_fit2, 'chi2_fit2/F' )
tree.Branch( 'chi2_fit3', chi2_fit3, 'chi2_fit3/F' )
tree.Branch( 'NDF_fit1', NDF_fit1, 'NDF_fit1/F' )
tree.Branch( 'NDF_fit2', NDF_fit2, 'NDF_fit2/F' )
tree.Branch( 'NDF_fit3', NDF_fit3, 'NDF_fit3/F' ) 
tree.Branch( 'prob_fit1', prob_fit1, 'prob_fit1/F' )
tree.Branch( 'prob_fit2', prob_fit2, 'prob_fit2/F' )
tree.Branch( 'prob_fit3', prob_fit3, 'prob_fit3/F' ) 
tree.Branch( 'N_fit1', N_fit1, 'N_fit1/F' )
tree.Branch( 'N_fit2', N_fit2, 'N_fit2/F' )
tree.Branch( 'N_fit3', N_fit3, 'N_fit3/F' ) 
tree.Branch( 'Nfit_fit1', Nfit_fit1, 'Nfit_fit1/F' )
tree.Branch( 'Nfit_fit2', Nfit_fit2, 'Nfit_fit2/F' )
tree.Branch( 'Nfit_fit3', Nfit_fit3, 'Nfit_fit3/F' ) 


tree.Branch( 'x0_dnn1', x0_dnn1, 'x0_dnn1/F' )
tree.Branch( 'x0_dnn2', x0_dnn2, 'x0_dnn2/F' )
tree.Branch( 'x0_dnn3', x0_dnn3, 'x0_dnn3/F' )
tree.Branch( 'Ax_dnn1', Ax_dnn1, 'A0_dnn1/F' )
tree.Branch( 'Ax_dnn2', Ax_dnn2, 'A0_dnn2/F' )
tree.Branch( 'Ax_dnn3', Ax_dnn3, 'A0_dnn3/F' ) 

print ("len(y_test) ",len(y_test))
for i in range(len(y_test)):

  if len(y_test[i])>0:
   x0_true1[0] = (y_test[i])[0][0]
   Ax_true1[0] = (y_test[i])[0][1]

   x0_fit1[0] = (y0_fit[i])[0][0]
   Ax_fit1[0] = (y0_fit[i])[0][1]

   chi2_fit1[0] = (y0_fit[i])[0][2]
   NDF_fit1[0]  = (y0_fit[i])[0][3]
   prob_fit1[0] = (y0_fit[i])[0][4]
   N_fit1[0]    = (y0_fit[i])[0][5]
   Nfit_fit1[0] = (y0_fit[i])[0][6]

   x0_dnn1[0] = (y_new[i])[0][0]
   Ax_dnn1[0] = (y_new[i])[0][1]

  if len(y_test[i])>1:
   x0_true2[0] = (y_test[i])[1][0]
   Ax_true2[0] = (y_test[i])[1][1]

   x0_fit2[0] = (y0_fit[i])[1][0]
   Ax_fit2[0] = (y0_fit[i])[1][1]

   chi2_fit2[0] = (y0_fit[i])[1][2]
   NDF_fit2[0]  = (y0_fit[i])[1][3]
   prob_fit2[0] = (y0_fit[i])[1][4]
   N_fit2[0]    = (y0_fit[i])[1][5]
   Nfit_fit2[0] = (y0_fit[i])[1][6]
 
   x0_dnn2[0] = (y_new[i])[1][0]
   Ax_dnn2[0] = (y_new[i])[1][1]

  if len(y_test[i])>2:
   x0_true3[0] = (y_test[i])[2][0]
   Ax_true3[0] = (y_test[i])[2][1]

   x0_fit3[0] = (y0_fit[i])[2][0]
   Ax_fit3[0] = (y0_fit[i])[2][1]

   chi2_fit3[0] = (y0_fit[i])[2][2]
   NDF_fit3[0]  = (y0_fit[i])[2][3]
   prob_fit3[0] = (y0_fit[i])[2][4]
   N_fit3[0]    = (y0_fit[i])[2][5]
   Nfit_fit3[0] = (y0_fit[i])[2][6]

   x0_dnn3[0] = (y_new[i])[2][0]
   Ax_dnn3[0] = (y_new[i])[2][1]


 
  tree.Fill()
  #tree.Print()

tree.Write()
outfile.Close()


#TF1 *xzfit = new TF1("xzfit","pol1",-50,2500);
#TGraphErrors *grxz = new TGraphErrors(nx,zhx,xh,zhx_err,xh_err);

#grxz->Fit(xzfit,"+robq");
#          double x0 = xzfit->GetParameter(0);
#          double Ax = xzfit->GetParameter(1);
#          double chi2x = xzfit->GetChisquare();
#          int ndfx = xzfit->GetNDF();
#          line linex(x0,Ax,0.0,0.0,0.0,chi2x,0.0,ndfx,0,0,0);

'''



In [0]:
def plot_predict(y_new = y_new):

  print("Plot fitted parameters, efficiency, prob_noise = ",efficiency,prob_noise)

  print("y_new shape :: ",y_new.shape)
  
  # plot fitted parameters
  y0_true = []
  y0_pred = []
  for i in range(len(x_test)):
    y0_true.append((y_test[i])[0])
    y0_pred.append((y_new[i])[0])
    if i<6:
        print ("true ",y_test[i])
        print ("pred ",y_new[i])
        #print (y0_true[i], y0_pred[i])
  plt.plot(y0_true, y0_pred, ".")
  plt.ylabel('predicted')
  plt.xlabel('true')
  print('params_scatter_'+str(efficiency)+'_'+str(prob_noise)+'.png')
  plt.savefig('params_scatter_'+str(efficiency)+'_'+str(prob_noise)+'.png')
 
  plt.show()
  plt.clf()


  # the histogram of the data
  columns_true = zip(*y0_true) #transpose rows to columns
  columns_pred = zip(*y0_pred) #transpose rows to columns
  #diff = columns_pred-columns_true
  diff = tuple(np.subtract(columns_pred,columns_true))

  plt.subplot(1, 2, 1)
  n, bins, patches = plt.hist(diff[0], 50, normed=1, facecolor='green', alpha=0.30)
  # Fit a normal distribution to the data:
  mu, std = norm.fit(diff[0])
  # Plot the PDF.
  xmin, xmax = plt.xlim()
  x = np.linspace(xmin, xmax, 100)
  p = norm.pdf(x, mu, std)
  plt.title("slope X: mu = %.2f,  std = %.2f" % (mu, std))
  plt.plot(x, p, 'k', linewidth=2)

  plt.subplot(1, 2, 2)
  n, bins, patches = plt.hist(diff[1], 50, normed=1, facecolor='green', alpha=0.30)
  # Fit a normal distribution to the data:
  mu, std = norm.fit(diff[1])
  # Plot the PDF.
  xmin, xmax = plt.xlim()
  x = np.linspace(xmin, xmax, 100)
  p = norm.pdf(x, mu, std)
  plt.title("offset X: mu = %.2f,  std = %.2f" % (mu, std))
  plt.plot(x, p, 'k', linewidth=2)

  print('params_1_'+str(efficiency)+'_'+str(prob_noise)+'.png')
  plt.savefig('params_1_'+str(efficiency)+'_'+str(prob_noise)+'.png')

  plt.show()
  plt.clf()

  plt.subplot(1, 2, 1)
  n, bins, patches = plt.hist(diff[2], 50, normed=1, facecolor='green', alpha=0.30)
  # Fit a normal distribution to the data:
  mu, std = norm.fit(diff[2])
  # Plot the PDF.
  xmin, xmax = plt.xlim()
  x = np.linspace(xmin, xmax, 100)
  p = norm.pdf(x, mu, std)
  plt.title("slope Y: mu = %.2f,  std = %.2f" % (mu, std))
  plt.plot(x, p, 'k', linewidth=2)

  plt.subplot(1, 2, 2)
  n, bins, patches = plt.hist(diff[3], 50, normed=1, facecolor='green', alpha=0.30)
  # Fit a normal distribution to the data:
  mu, std = norm.fit(diff[3])
  # Plot the PDF.
  xmin, xmax = plt.xlim()
  x = np.linspace(xmin, xmax, 100)
  p = norm.pdf(x, mu, std)
  plt.title("offset Y: mu = %.2f,  std = %.2f" % (mu, std))
  plt.plot(x, p, 'k', linewidth=2)


  print('params_2_'+str(efficiency)+'_'+str(prob_noise)+'.png')
  plt.savefig('params_2_'+str(efficiency)+'_'+str(prob_noise)+'.png')


  plt.show()
  plt.clf()

  



In [0]:

def plot_predict_event(y_new = y_new):

  #plot sample events with true and reconstructed tracks
  print("Plot events with predicted tracks, efficiency, prob_noise = ",efficiency,prob_noise)
  
  print("y_new shape :: ",y_new.shape)
  

  for n in range(8):
    for k in range(int(100*(prob_noise+0.10))):
      rnumber = np.random.randint(0,len(x_test))
      x,y,z = x_test[rnumber,0].nonzero()
      from mpl_toolkits.mplot3d import Axes3D
      fig = plt.figure()
      ax = fig.add_subplot(111, projection='3d')
      ax.scatter(x, y, z, zdir='z', c= 'red', s=5, alpha=0.9)
    
      ax.set_xlabel('X')
      ax.set_ylabel('Y')
      ax.set_zlabel('Z (depth)')
  
  
   
    
      for j in range(len(y_test[rnumber])):
      # Data for a three-dimensional line
         #print("index ",k)
         #print("ytest ", y_test[rnumber])
         zline = np.linspace(0, 28, 100)
         xline = y_test[rnumber,j,0]*det_width+y_test[rnumber,j,1]*slope_mult*zline 
         yline = y_test[rnumber,j,2]*det_width+y_test[rnumber,j,3]*slope_mult*zline 
###       xline = 100*y_test[rnumber,k,0]+100/slope_scale*y_test[rnumber,k,1]*zline
###       yline = 100*y_test[rnumber,k,2]+100/slope_scale*y_test[rnumber,k,3]*zline
         ax.plot3D(xline, yline, zline,  'green', alpha=0.5)
         xline = y_new[rnumber,j,0]*det_width+y_new[rnumber,j,1]*slope_mult*zline 
         yline = y_new[rnumber,j,2]*det_width+y_new[rnumber,j,3]*slope_mult*zline  
###       xline = 100*y_new[rnumber,k,0]+100/slope_scale*y_new[rnumber,k,1]*zline
###       yline = 100*y_new[rnumber,k,2]+100/slope_scale*y_new[rnumber,k,3]*zline
         ax.plot3D(xline, yline, zline,  'blue', alpha=0.5)
    
      #ax.view_init(elev=10., azim=0)
      ax.scatter([0,det_width],[0,det_width],[0,det_depth], c= 'blue', s=0)

      # set axes range
      ax.set_xlim3d(0,det_width)
      ax.set_xlim3d(0,det_width)
      ax.set_xlim3d(0,det_depth)
      #ax.axis('equal')

      print('event_res_'+str(efficiency)+"_"+str(prob_noise)+"_"+str(k)+'.png')
      plt.savefig('event_res_'+str(efficiency)+"_"+str(prob_noise)+"_"+str(k)+'.png')


      
      plt.show()
  


In [0]:
def write_predict(y_new = y_new):

  print("Write output data, efficiency, prob_noise = ",efficiency,prob_noise)

  print("y_new shape :: ",y_new.shape)
  

  # write the output - true and reconstructed tracks	
  print("Writing dnn_output.txt by printing y_new and y_test: ",y_new.shape)  
  f= open("dnn_output"+str(efficiency)+"_"+str(prob_noise)+".txt","w+")
  for i in range(len(y_test)):
    k = len(y_test[i])
    f.write("%s \n" % k)
    for j in range(len(y_test[i])):
      for l in range(len(y_test[i][j])):
        f.write("%s " % y_test[i][j][l])
      f.write("\n")
    for j in range(len(y_new[i])):
      for l in range(len(y_new[i][j])):
        f.write("%s " % y_new[i][j][l])      
      f.write("\n")
    
  f.close()





In [0]:

print("number of tracks ",mean_tracks)



for prob_noise in [0, 0.050, 0.100, 0.150, 0.200]:
#for prob_noise in [0.001, 0.004]:
  
  train_model(model, prob_noise)
  

  x_test, y_test = generate_testdata(prob_noise)
  plot_sample()
  
  y_new = predict(model)
  print("y_new shape: ",y_new.shape)
  plot_predict(y_new)
  plot_predict_event(y_new)
  
  write_predict(y_new)
  
  !zip  /content/dnn_output_0.zip /content/*.png /content/*.txt

In [0]:


end_training_time = time.time()



!zip  /content/dnn_output_final.zip /content/*.png /content/*.txt



print ("    ")
print ("###########################################################")
print ("    ")
print ("CPU usage: ")
print ("Deep NN training:      ", end_training_time-start_training_time)
#print ("Pattern recognition:   ", end_pattern_time-start_pattern_time)
#print ("Track fitting:         ", end_fit_time-start_fit_time)
print ("    ")
print ("###########################################################")
print ("    ")

